In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
# from PIL import Image

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import initializers

# Keras utilities
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#from tensorflow.keras.optimizers import Adam




In [ ]:
from files import utils

In [ ]:
"""
Simple example network for visualising the filters and layers.
"""

num_classes = 2
image_size = 8

num_filters = 8 # also 4, 16
num_filters_l2 = 16
filter_size = 3 # also 4,5
# num_hidden = 64 # also 16?)

# batch_size = 5 # Half the images sent in each mini-batch
#batch_size = 10 # All images in each batch
batch_size = 1 # One at a time
# batch_size = 2 # 2 at a time

# initializer = initializers.Constant(0.1)
kernel_init = initializers.RandomUniform(minval=-0.01, maxval=0.01)

EPOCHS = 100

data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.5, interpolation='nearest', fill_mode='constant', fill_value=0.0),
    ]
)

model = Sequential(
    [
        layers.Input((image_size, image_size, 3)),
        data_augmentation,
        layers.Rescaling(1.0 / 255),                       # Rescale to 0..1
        layers.Conv2D(num_filters, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init),
        layers.MaxPooling2D(), # Halve the resolution to 4x4
        layers.Conv2D(num_filters_l2, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init),
        layers.MaxPooling2D(), # Halve the resolution to 2x2
        # layers.Conv2D(num_filters, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init),
        # layers.MaxPooling2D(),
        # layers.Dropout(0.2),
        layers.Flatten(),                                                                       # Flatten filters into a 1D vector of vars
        ##layers.Dense(num_hidden, activation="relu"),                                            # Hidden layer
        layers.Dense(num_classes, activation="softmax", kernel_initializer=kernel_init),                                        # output layer
    ]
)


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary(line_length=80)

In [ ]:
utils.report_weights(model)

In [ ]:
# Do some training

# data_dir = "images_noisy_RGB_train"
# train_data_dir = "images_clean/train"  # Clean training data
# train_data_dir = "images_noisy_GB_train/train"  # Random FP in training set
data_dir = "files/images_red"  # Single-channel (red) images

train_data_dir = data_dir + "/train"
valid_data_dir = data_dir + "/val"
test_data_dir = data_dir + "/test"

train_generator = tf.keras.utils.image_dataset_from_directory(train_data_dir, image_size=(image_size,image_size), batch_size = batch_size)
valid_generator = tf.keras.utils.image_dataset_from_directory(valid_data_dir, image_size=(image_size,image_size), batch_size = batch_size)
    
# model.fit_generator(train_generator, epochs=fine_tune_epochs, validation_data=valid_generator, callbacks = callbacks)
model.fit(train_generator, validation_data=valid_generator, epochs=EPOCHS, shuffle=True)

In [ ]:
# Test the model
test_images = os.listdir(test_data_dir)
class_names = ['X', '0']
tot_correct = 0
for image_file in test_images:
    img = tf.keras.utils.load_img(os.path.join(test_data_dir, image_file), target_size=(image_size, image_size))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    predictions = model.predict(img_array)
    scores = tf.nn.softmax(predictions[0])
    score = np.max(scores)
    ans = class_names[np.argmax(scores)]
    correct = (ans == image_file[0]) # Filename is prefixed with class
    tot_correct += correct
    print(f"{image_file}: {ans} {score} {correct}")
print("==================================")
print(f"{tot_correct} of {len(test_images)} correct.")

In [ ]:
print("========================= TRAINED WEIGHTS ===============================")
utils.report_weights(model)

In [ ]:
# Example "X"
utils.report_outputs(model, os.path.join(test_data_dir, test_images[8]), image_size)

In [ ]:
# Example "O"
utils.report_outputs(model, os.path.join(test_data_dir, test_images[4]), image_size)